In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import logging

logger = logging.getLogger('')
logger.setLevel(logging.INFO)


In [2]:
class Model(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        p_hidden_size = 100
        l_hidden_size = 100
        hidden_size = 100

        hdim = obs_dim // (act_dim+1)

        self.participant_branch = nn.Sequential(
            nn.Linear(hdim * act_dim, p_hidden_size),
            nn.ReLU()
        )

        self.loss_branch = nn.Sequential(
            nn.Linear(hdim, l_hidden_size),
            nn.ReLU()
        )

        self.merge = nn.Sequential(
            nn.Linear(p_hidden_size + l_hidden_size, hidden_size),
            nn.ReLU()
        )
    
    def transfer(self, x):
        p = x[..., :-1]
        l = x[..., -1:]
        return p.reshape(p.size(0), -1), l.reshape(l.size(0), -1)
    def forward(self, x):
        p, l = self.transfer(x)
        pb = self.participant_branch(p)
        lb = self.loss_branch(l)
        x = torch.cat([pb, lb], dim=-1)
        return self.merge(x)

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ResidualBlock, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        residual = x
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x += residual  # 跳跃连接
        x = self.relu(x)
        return x

class ModelRes(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        p_hidden_size = 50
        l_hidden_size = 50
        hidden_size = 50
        num_blocks = 2
        hdim = obs_dim // (act_dim+1)

        self.participant_branch = nn.Sequential(
            nn.Linear(hdim * act_dim, p_hidden_size),
            nn.ReLU()
        )
        self.loss_branch = nn.Sequential(
            nn.Linear(hdim, l_hidden_size),
            nn.ReLU()
        )
        self.merge = nn.Sequential(
            nn.Linear(p_hidden_size + l_hidden_size, hidden_size),
            nn.ReLU()
        )
        self.res_blocks = nn.Sequential(*[ResidualBlock(hidden_size, hidden_size) for _ in range(num_blocks)])
        self.net = nn.Sequential(
            self.merge,
            self.res_blocks,
            nn.Linear(hidden_size, act_dim),
            # nn.Softmax(dim=1),
        )
    def transfer(self, x):
        p = x[..., :-1]
        l = x[..., -1:]
        return p.reshape(p.size(0), -1), l.reshape(l.size(0), -1)
    def forward(self, obs):
        p, l = self.transfer(obs)
        pb = self.participant_branch(p)
        lb = self.loss_branch(l)
        obs = torch.cat([pb, lb], dim=-1)
        return self.net(obs)

class ModelResSoft(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        p_hidden_size = 50
        l_hidden_size = 50
        hidden_size = 50
        num_blocks = 2
        hdim = obs_dim // (act_dim+1)

        self.participant_branch = nn.Sequential(
            nn.Linear(hdim * act_dim, p_hidden_size),
            nn.ReLU()
        )
        self.loss_branch = nn.Sequential(
            nn.Linear(hdim, l_hidden_size),
            nn.ReLU()
        )
        self.merge = nn.Sequential(
            nn.Linear(p_hidden_size + l_hidden_size, hidden_size),
            nn.ReLU()
        )
        self.res_blocks = nn.Sequential(*[ResidualBlock(hidden_size, hidden_size) for _ in range(num_blocks)])
        self.net = nn.Sequential(
            self.merge,
            self.res_blocks,
            nn.Linear(hidden_size, act_dim),
            nn.Softmax(dim=1),
        )
    def transfer(self, x):
        p = x[..., :-1]
        l = x[..., -1:]
        return p.reshape(p.size(0), -1), l.reshape(l.size(0), -1)
    def forward(self, obs):
        p, l = self.transfer(obs)
        pb = self.participant_branch(p)
        lb = self.loss_branch(l)
        obs = torch.cat([pb, lb], dim=-1)
        return self.net(obs)

In [4]:
from helper import Camera
import pickle
import scipy

In [6]:
def spearmanFromActProb(rank, act_prob):
    x = [act_prob[i] for i in rank]
    y = [c for c in range(len(rank))]
    return scipy.stats.spearmanr(x,y)[0]
def spearmanSequenceFromActProbList(rank, act_probList):
    cos = []
    for act_prob in act_probList:
        cos.append(spearmanFromActProb(rank, act_prob))
    return np.array(cos)
def evaluateContributionsFromLogs(logs, client_nums):
    contirbution = []
    ddl = len(logs)
    for r in range(ddl):
        contirbution.append([0 for c in range(client_nums)])
    pprev_acc = logs[0].acc
    prev_participants = logs[1].participants
    prev_acc = logs[1].acc
    for r in range(2, ddl):
        for c in range(client_nums):
            contirbution[r][c] = contirbution[r-1][c]
        participants, acc = logs[r].participants, logs[r].acc
        if acc - prev_acc > prev_acc - pprev_acc:
            for c in prev_participants:
                contirbution[r][c] -= 1
            for c in participants:
                contirbution[r][c] += 1
        
        if acc < prev_acc:
            for c in participants:
                contirbution[r][c] -= 1
    return contirbution

In [45]:
with open('camera/cn5_pn3_dFMNIST_aMLP_piid_seed2.camera', 'rb') as f:
    camera = pickle.load(f)
curr_round = 4
act_probs = camera.act_probLogs[curr_round]
contributions = evaluateContributionsFromLogs(camera.getRpmLogs(curr_round), camera.client_nums)
sco_rl = spearmanSequenceFromActProbList(camera.rank, act_probs)
sco_qi = spearmanSequenceFromActProbList(camera.rank, contributions)
sco_rl[199], sco_qi[199]

(0.5574917491749174, 0.034013848222753246)

In [17]:
sco_rl[199], sco_qi[199]

(0.4913171317131712, 0.4146991669867845)

In [32]:
log = rpm.buffer[r]
log.acc, log.loss, log.participants

(0.4072857142857143,
 0.058279946225030084,
 array([37, 33, 36, 39,  0, 56, 12, 24,  3, 99]))

In [41]:

logs = [rpm.buffer[i] for i in range(8, 209)]
contributions = evaluateContributionsFromLogs(logs, camera.client_nums)

In [53]:

s1 = spearmanFromActProb(camera.rank, contributions[199])
s2 = spearmanFromActProb(camera.rank, camera.act_probLogs[curr_round][199])
s1, s2

(0.4146991669867845, 0.48400840084008395)

In [14]:
bs = 2
act_dim = 3
h_dim = 2
obs_dim = (act_dim + 1) * h_dim
print(obs_dim)
m1 = ModelRes(obs_dim=obs_dim, act_dim=act_dim)
m2 = ModelResSoft(obs_dim=obs_dim, act_dim=act_dim)

8


In [24]:
x = torch.randn([bs,1 , h_dim, act_dim+1])
p = m1(x)
soft1 = nn.Softmax(dim=1)
soft2 = F.softmax
p1 = soft1(p)
p2 = soft2(p, dim=-1)
p1, p2

(tensor([[0.2650, 0.4232, 0.3119],
         [0.2331, 0.4504, 0.3166]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.2650, 0.4232, 0.3119],
         [0.2331, 0.4504, 0.3166]], grad_fn=<SoftmaxBackward0>))

In [17]:
F.softmax(p1,dim=1)

tensor([[0.2851, 0.4292, 0.2857],
        [0.3004, 0.3787, 0.3209]], grad_fn=<SoftmaxBackward0>)

In [27]:
l = [i for i in range(0, 32)]

In [28]:
l[-64:]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [8]:
f = nn.BCEWithLogitsLoss(reduction='none')
g = nn.BCEWithLogitsLoss()

In [9]:
f(pred, one_hots), g(pred, one_hots)

(tensor([[0.7762, 0.7756, 0.8213],
         [0.6489, 0.6156, 0.8133]],
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.7418, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>))

In [13]:
t = f(pred, one_hots)

tensor([[0.7762, 0.7756, 0.8213],
        [0.6489, 0.6156, 0.8133]],
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [14]:
rewards

tensor([[-0.9001],
        [-0.2279]])

In [15]:
t * rewards

tensor([[-0.6986, -0.6981, -0.7392],
        [-0.1479, -0.1403, -0.1853]], grad_fn=<MulBackward0>)

In [86]:
tensor = torch.randn(2, 1, 2, 4)  # 假设形状为[2, 1, 3, 4]

# 将张量形状变为[a, c * d]
reshaped_tensor = tensor.view(tensor.size(0), -1)
reshaped_tensor.shape

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [5]:
x = torch.randn([4, 26])
x.unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 4, 101])

In [19]:
192/32

6.0

In [22]:
x = torch.randn([2,1,4,26])
model(x)

tensor([[0.0063, 0.0088, 0.0052, 0.0104, 0.0117, 0.0149, 0.0042, 0.0177, 0.0145,
         0.0126, 0.0185, 0.0101, 0.0089, 0.0083, 0.0085, 0.0071, 0.0065, 0.0096,
         0.0076, 0.0090, 0.0086, 0.0087, 0.0123, 0.0064, 0.0098, 0.0198, 0.0127,
         0.0047, 0.0045, 0.0122, 0.0080, 0.0050, 0.0077, 0.0074, 0.0092, 0.0072,
         0.0068, 0.0070, 0.0136, 0.0060, 0.0074, 0.0058, 0.0151, 0.0072, 0.0059,
         0.0175, 0.0125, 0.0095, 0.0078, 0.0152, 0.0197, 0.0080, 0.0052, 0.0149,
         0.0094, 0.0080, 0.0137, 0.0151, 0.0110, 0.0049, 0.0029, 0.0084, 0.0132,
         0.0051, 0.0131, 0.0140, 0.0225, 0.0103, 0.0127, 0.0049, 0.0108, 0.0102,
         0.0099, 0.0047, 0.0082, 0.0087, 0.0043, 0.0092, 0.0064, 0.0075, 0.0113,
         0.0117, 0.0096, 0.0129, 0.0056, 0.0069, 0.0111, 0.0087, 0.0215, 0.0091,
         0.0065, 0.0239, 0.0126, 0.0122, 0.0142, 0.0099, 0.0122, 0.0078, 0.0092,
         0.0043],
        [0.0059, 0.0099, 0.0168, 0.0112, 0.0080, 0.0091, 0.0171, 0.0086, 0.0055,
         0

In [14]:
input_size = 4 * 101
kernel_size = 3
padding = 1
stride = 1
(input_size - kernel_size + 2 * padding) / stride + 1

404.0

800